<a href="https://colab.research.google.com/github/saurav-kothale/WebsiteBot-Llama-3-GenAI/blob/main/Website_bot_using_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate transformers
!pip -q install sentence_transformers
!pip -q install unstructured
!pip install pinecone

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata

pc = Pinecone(api_key=userdata.get('pinecone_key'))


In [ ]:
pip install -U langchain-community

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone as PineconeStore
import pinecone

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

In [ ]:
!pip install numpy
!pip install python-magic
!pip install --upgrade nltk

In [ ]:
Urls = ["https://ineuron.ai/"]

loader = UnstructuredURLLoader(urls = Urls)
data = loader.load()

data

In [ ]:
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size = 1000,
    chunk_overlap = 200
)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

6

In [ ]:
text_chunks[0]

Document(metadata={'source': 'https://ineuron.ai/'}, page_content='Learning with iNeuron made\nTake your career to the next level with industry ready programs, An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster. Experience a tech community like never seen before\nOur Courses\nSuccess Stories\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nSoftware Engineer - Python')

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

384

In [ ]:
index_name = "website-bot"

pc.create_index(
    name = index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
  )
)

index = pc.Index(index_name)

In [ ]:
index_name = "website-bot"
index = pc.Index(index_name)

In [ ]:
!pip install --upgrade langchain-pinecone
!pip install langchain-pinecone

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store.add_documents(documents=text_chunks)

In [ ]:
notebook_login()

In [ ]:
!pip -qq install langchain Groq langchain_core langchain_groq streamlit

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain

# prompt_template = PromptTemplate(
#     input_variables=["course_name"],
#     template="What is the price of the {course_name} course?"
# )

In [ ]:
pipe = transformers.pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                temperature=0.8,
                top_p=0.95,
                repetition_penalty=1.2,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe)

In [ ]:
# llm_chain = LLMChain(llm = llm, prompt = prompt_template)

In [ ]:
# course_name = "Full Stack Data Analytics"
# response = llm_chain.run(course_name)

In [ ]:
llm.predict("Please provide a concise summary of the Book Harry Potter")

In [ ]:
from langchain import PromptTemplate

prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a precise and accurate AI assistant. Your task is to answer the given question based solely on the provided context. If the information is not present in the context, respond with "The provided context does not contain any information about this topic."
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Context: {context}
Question: {query}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

# Initialize the LangChain PromptTemplate
template = PromptTemplate(
    input_variables=["query"],
    template=prompt_template,
)

In [ ]:
!pip install --upgrade langchain

In [ ]:
from langchain.chains import RetrievalQA
from langchain_core.output_parsers import StrOutputParser

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3}, chain_type_kwargs = {"prompt": template,"output_parser": StrOutputParser()}))

In [ ]:
query = "Is INeuron has hiring partners?"

In [ ]:
result = qa({"query": query})

In [ ]:
answer = result['result']
print(answer)

In [ ]:
def post_process_answer(answer):
    # Remove any text after "FINAL ANSWER:" if present
    if "FINAL ANSWER:" in answer:
        answer = answer.split("FINAL ANSWER:")[1].strip()

    # Remove any remaining instructions or prefixes
    answer = answer.replace("Answer:", "").strip()

    return answer

# When using the QA system
query = "Is INeuron has hiring partners?"
result = qa.run(query)
processed_answer = post_process_answer(result)
print(processed_answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


This Agreement is governed by English law.

QUESTION: What did the president say about Michael Jackson?
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, the